In [ ]:

from IPython.display import Markdown, display

from wid_config import WIDConfig
from wid_torch import *

import pandas as pd
import numpy as np
np.random.seed(42)
rng = np.random.default_rng(42)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from pathlib import Path
data_path = Path('./data')
data_dir = data_path / 'train_images'


import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

torch.manual_seed(2024)

## Load Data

In [ ]:
config = WIDConfig(verbose=True, model_name="resnet18")

# config.df = config.df.sample(n=1000, random_state=config.random_state)

## Instantiate wid_torch for PyTorch model 1

## define the model

In [ ]:
# base model
_model = models.resnet18(weights="ResNet18_Weights.IMAGENET1K_V1")

# rebuild the head to suit our purpose
# strucure 2048 --> 32 --> num_classes
_model.fc = nn.Sequential(
    nn.Linear(_model.fc.in_features, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(32, config.num_classes),
)
wid_model = WIDTorch(config=config, model=_model)

In [ ]:
wid_model.train(freeze=20)

In [ ]:
wid_model.evaluate_holdout()

In [ ]:
wid_model.train(freeze=False)

In [ ]:
wid_model.evaluate_holdout()

In [ ]:
wid_model.train(freeze=20)

In [ ]:
wid_model.plot_losses()

In [ ]:
wid_model.plot_scores()

## Load the best model and make predictions on the test set

### Early Stopped model

In [ ]:
wid_model.resume(_model, "Exp:14_Model:resnet18_epoch:45_vloss:0.5642_auc:0.7732.pth")
wid_model.evaluate_holdout()

### Best Validation Loss Model

In [ ]:
wid_model.resume(_model, "Exp:14_Model:resnet18_epoch:40_vloss:0.5588_auc:0.7769_L.pth")
wid_model.evaluate_holdout()

In [ ]:
_model